In [3]:
import requests
import json
from typing import Dict, Any

In [10]:
app_name = 'test_task'
openrouter_api_key = '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'

In [27]:
model = 'google/gemini-2.0-flash-001'

In [28]:
def make_request(prompt: str) -> Dict[str, Any]:
    response = requests.post(
      url="https://openrouter.ai/api/v1/chat/completions",
      headers={
        "Authorization": f"Bearer {openrouter_api_key}",
        "HTTP-Referer": app_name, # Optional. Site URL for rankings on openrouter.ai.
        "X-Title": app_name, # Optional. Site title for rankings on openrouter.ai.
      },
      data=json.dumps({
        "model": model,
        "messages": [
          {
            "role": "user",
            "content": prompt,
          }
        ],
        "max_tokens": 1000,
      })
    )

    res = response.json()
    return res

In [33]:
def improve_tweet(tweet_data: Dict[str, Any]) -> Dict[str, Any]:
    
    prompt = """You will be given tweet data in tags <tweet_data>. 

You should improve text.
- Suggest content improvements based on high-performing tweet patterns.
- Optimize phrasing: Recommend changes in wording or tone.
- Hashtag recommendations: Suggest effective hashtags.
- Call-to-action enhancements: Improve engagement triggers (e.g., “Retweet if you agree!”).

The best posting time is
- Days: Monday, Thursday, Friday 
- Hours: 9, 10, 11, 14, 16
- Depending on tweet data recommend next time closest day and hour for posting

Return result in json format:
improved_text: str - new improved text with emojis, call-to-action
hash_tags: str - effective hashtags for text
next_best_posting_time: str - closest day and hour from best posting time after the day-time from tweet_data.

Example:
<tweet_data>
{
  "Text": "New job opening at our company! Apply here: https://link.com #hiring #jobs",
  "Weekday": "Monday",
  "Hour": 15
}
</tweet_data>

Output:
{
"improved_text": "Exciting opportunity! 🚀 We're hiring for a new role – apply today! 🌟 → [link](https://link.com)",
"hash_tags": "#JobSearch #CareerGrowth",
"next_best_posting_time": "Monday, Hour: 16"
}

Generate only JSON data without any markup or other infornation!

<tweet_data><text></tweet_data>"""
    prompt = prompt.replace('<text>', json.dumps(tweet_data, ensure_ascii=False, indent=1))

    resp = make_request(prompt)
    res = resp['choices'][0]['message']['content']
    res = res.replace('```json', '')
    res = res.replace('```', '')
    try:
        data = json.loads(res)
    except:
        data = {}
    return data

In [34]:
data = {'Text': "Apple reportedly defects iCloud from AWS to Google Cloud: Apple's use of Google Cloud Platform which began wi... https://www.google.com/url?rct=j&sa=t&url=http://thedailycroton.com/2016/03/20/apple-reportedly-defects-icloud-from-aws-to-google-cloud/&ct=ga&cd=CAIyGjNhNjYxYTQyZWZjNWIxNWM6Y29tOmVuOlVT&usg=AFQjCNFUM2Cm31Guqe6MJ8vYA7qKGZGgYA&utm_source=twitterfeed&utm_medium=twitter",
        'Weekday': 'Tuesday', 
        'Hour': 6
       }

In [35]:
res = improve_tweet(data)

In [36]:
res

{'improved_text': 'Apple is reportedly moving iCloud from AWS to Google Cloud! ☁️ Big shift in cloud strategy. 🤔 Read more here: [link](https://www.google.com/url?rct=j&sa=t&url=http://thedailycroton.com/2016/03/20/apple-reportedly-defects-icloud-from-aws-to-google-cloud/&ct=ga&cd=CAIyGjNhNjYxYTQyZWZjNWIxNWM6Y29tOmVuOlVT&usg=AFQjCNFUM2Cm31Guqe6MJ8vYA7qKGZGgYA&utm_source=twitterfeed&utm_medium=twitter) What do you think about this change? Share your thoughts! 👇',
 'hash_tags': '#Apple #iCloud #GoogleCloud #AWS #CloudComputing',
 'next_best_posting_time': 'Thursday, Hour: 9'}